# Photoacoustic Computed Tomography (PACT) Reconstruction
## Key Steps
- Data Acquisition
- Signal Processing
- Image Reconstruction
- Visualization

In [ ]:
# %% [markdown]
# # PACT Data Generation
# Generate synthetic breast phantoms and simulate measurements

# %%
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import gaussian_filter
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# %%
def generate_breast_phantom(grid_size=(256, 256), breast_type='D'):
    """Generate simplified breast phantom for testing"""
    nx, ny = grid_size
    
    # Create coordinate grid
    x = np.linspace(-1, 1, nx)
    y = np.linspace(-1, 1, ny)
    X, Y = np.meshgrid(x, y, indexing='ij')
    R = np.sqrt(X**2 + Y**2)
    
    # Breast shape (ellipse)
    theta = 0.3  # Rotation
    a, b = 0.6, 0.8  # Semi-axes
    breast_mask = ((X*np.cos(theta) + Y*np.sin(theta))**2 / a**2 +
                  (X*np.sin(theta) - Y*np.cos(theta))**2 / b**2) <= 1
    
    # Number of structures based on breast type
    n_structures = {'A': 3, 'B': 8, 'C': 15, 'D': 25}[breast_type]
    
    # Generate initial pressure (optical absorption)
    p0 = np.zeros((nx, ny))
    for _ in range(n_structures):
        cx, cy = np.random.uniform(-0.5, 0.5, 2)
        r = np.random.uniform(0.05, 0.15)
        intensity = np.random.uniform(0.3, 1.0)
        
        # Gaussian-shaped structure
        dist = np.sqrt((X - cx)**2 + (Y - cy)**2)
        p0 += intensity * np.exp(-dist**2 / (2*r**2))
    
    # Apply breast mask
    p0 = p0 * breast_mask
    p0 = (p0 - p0.min()) / (p0.max() - p0.min())
    
    # Speed of sound distribution
    c_base = 1540.0  # m/s
    sos_factors = {'A': 0.94, 'B': 0.96, 'C': 0.98, 'D': 1.0}
    factor = sos_factors[breast_type]
    
    # Add heterogeneity correlated with pressure
    c_variation = 0.1 * p0
    c = c_base * (factor + c_variation) * breast_mask + c_base * (~breast_mask)
    
    return p0, c, breast_mask

# %%
# Generate phantom
grid_size = (128, 128)  # Reduced for faster computation
p0_true, c_true, mask = generate_breast_phantom(grid_size, 'D')

# %%
def forward_simulation_simple(p0, c, dx=0.32e-3, n_sensors=64, n_steps=200):
    """Simplified forward simulation for testing"""
    nx, ny = p0.shape
    
    # Create circular sensor array
    radius = 0.072  # 72mm
    angles = np.linspace(0, 2*np.pi, n_sensors, endpoint=False)
    sensors = np.column_stack([radius * np.cos(angles), radius * np.sin(angles)])
    
    # Initialize pressure field
    p = p0.copy()
    
    # Wave propagation parameters
    dt = dx / (np.sqrt(2) * np.max(c)) * 0.3  # CFL condition
    
    # Store measurements
    measurements = np.zeros((n_steps, n_sensors))
    
    # Simple wave propagation (2D diffusion-like approximation)
    for t in range(n_steps):
        # Simplified wave propagation using convolution
        kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]]) / (dx**2)
        laplacian = signal.convolve2d(p, kernel, mode='same', boundary='symm')
        
        # Update pressure (wave equation: d²p/dt² = c² ∇²p)
        if t == 0:
            p_prev = p.copy()
            p = p + 0.5 * (c**2) * (dt**2) * laplacian
        else:
            p_new = 2*p - p_prev + (c**2) * (dt**2) * laplacian
            p_prev, p = p, p_new
        
        # Sample at sensor locations
        for i, (x, y) in enumerate(sensors):
            xi = int((x / dx) + nx // 2)
            yi = int((y / dx) + ny // 2)
            if 0 <= xi < nx and 0 <= yi < ny:
                measurements[t, i] = p[xi, yi]
    
    return measurements, dt

# %%
# Simulate measurements
print("Simulating measurements...")
measurements, dt = forward_simulation_simple(p0_true, c_true, n_steps=200)

# Add noise
snr_db = 20
signal_power = np.mean(measurements**2)
noise_power = signal_power / (10**(snr_db/10))
noise = np.random.normal(0, np.sqrt(noise_power), measurements.shape)
measurements_noisy = measurements + noise

print(f"Measurement shape: {measurements.shape}")
print(f"Time step: {dt*1e6:.2f} μs")
print(f"SNR: {snr_db} dB")

# %%
# Save data
import os
os.makedirs('data', exist_ok=True)
np.savez('data/simulated_data.npz',
         p0_true=p0_true,
         c_true=c_true,
         mask=mask,
         measurements=measurements_noisy,
         dt=dt,
         dx=0.32e-3,
         snr_db=snr_db)

# %%
# Visualize
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

im1 = axes[0].imshow(p0_true.T, cmap='hot', origin='lower')
axes[0].set_title('Initial Pressure')
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(c_true.T, cmap='viridis', origin='lower')
axes[1].set_title('Speed of Sound (m/s)')
plt.colorbar(im2, ax=axes[1])

axes[2].imshow(mask.T, cmap='gray', origin='lower')
axes[2].set_title('Breast Mask')

# Plot sample sensor data
time = np.arange(200) * dt * 1e6
axes[3].plot(time, measurements_noisy[:, 0], label='Sensor 0', alpha=0.7)
axes[3].plot(time, measurements_noisy[:, 16], label='Sensor 16', alpha=0.7)
axes[3].set_xlabel('Time (μs)')
axes[3].set_ylabel('Pressure (arb.)')
axes[3].set_title('Sensor Measurements')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data/phantom_visualization.png', dpi=150, bbox_inches='tight')
plt.show()